<a href="https://colab.research.google.com/github/knoriy/depth_estimation/blob/master/model_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as vutils

In [0]:
# import os
# import time
# import tqdm
# import shutil
# import imageio
# import PIL.Image
# import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
# import IPython.display as display

## Globals

In [0]:
EPOCHS = 1000  # The number of itteration for training

LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0

IMG_WIDTH = IMG_HEIGHT = 28

In [0]:
SOURCE_DATA_DIR = "/content/Data"
CHECKPOINT_DIR  = "/content/checkpoint"
OUTPUT_DIR      = "/content/output"

# Utils

In [0]:
def im_show(tensor):
  if len(tensor.shape) == 4:
    tensor_image = image.view(image.shape[2], image.shape[3], image.shape[1])
    plt.imshow(tensor_image.detach().numpy() )
    plt.show()
  else:
    tensor_image = tensor.view(tensor.shape[1], tensor.shape[2], tensor.shape[0])
    plt.imshow(tensor_image.detach().numpy() )
    plt.show()

# Model

In [0]:
class Model(nn.Module):
  def __init__(self, num_channels=3):
    super(Model, self).__init__()
    self.relu = nn.ReLU()
    self.Dropout2d = nn.Dropout2d(0.1)
    self.MaxPool2d = nn.MaxPool2d(3, 1)
    self.BatchNorm2d = nn.BatchNorm2d
    self.conv1 = self.conv_block

    self.num_channels = num_channels

  def conv_block(self, x, in_channels, out_channels, kernel_size, stride=1, padding_mode='zeros'):
    x = nn.Conv2d(in_channels=in_channels, 
                  out_channels=out_channels, 
                  kernel_size=kernel_size, 
                  stride=stride, 
                  padding_mode=padding_mode, 
                  padding=1)(x)

    # x = self.BatchNorm2d(x)
    x = self.MaxPool2d(x)
    x = self.Dropout2d(x)
    x = self.relu(x)
    return x

  # Decoder
  def deconv_block(self, x, in_channels, out_channels, kernel_size, stride=1, padding_mode='zeros'):
    x = nn.ConvTranspose2d(in_channels=in_channels, 
                           out_channels=out_channels, 
                           kernel_size=kernel_size, 
                           stride=stride, 
                           padding_mode=padding_mode, 
                           padding=1)(x)

    # x = self.BatchNorm2d(x)
    x = self.relu(x)
    x = self.Dropout2d(x)
    return x

  # Residual
  def identity_block(self, x, in_channels, out_channels, stride=1, kernel_size=2, padding_mode='zeros'):

    x_shortcut = x

    x = nn.Conv2d(in_channels=in_channels, 
                  out_channels=out_channels, 
                  kernel_size=kernel_size, 
                  stride=stride, 
                  padding_mode=padding_mode)(x)

    # x = self.BatchNorm2d(x)
    x = self.relu(x)
    x = self.Dropout2d(x)

    x = nn.Conv2d(in_channels=in_channels, 
                  out_channels=out_channels, 
                  kernel_size=kernel_size, 
                  stride=stride, 
                  padding_mode=padding_mode)(x)

    # x = self.BatchNorm2d(x)
    x = self.relu(x)
    x = self.Dropout2d(x)

    x = nn.Conv2d(in_channels=in_channels, 
                  out_channels=out_channels, 
                  kernel_size=kernel_size, 
                  stride=stride, 
                  padding_mode=padding_mode)(x)

    # x = self.BatchNorm2d(x)
    x = torch.add(x, x_shortcut)
    x = self.relu(x)

    return x

  def forward(self, x):
    #####################################################################
    # Encoder
    #####################################################################
    x = self.conv_block(x=x, in_channels=3  , out_channels=64 , kernel_size=2, stride=1, padding_mode='replicate') 
    x = self.conv_block(x=x, in_channels=64 , out_channels=128, kernel_size=2, stride=2, padding_mode='replicate')
    x = self.conv_block(x=x, in_channels=128, out_channels=256, kernel_size=2, stride=2, padding_mode='replicate')
    # #####################################################################
    # # Encoder
    # #####################################################################



    # #####################################################################
    # # Residual Block
    # #####################################################################
    # x = self.identity_block(x=x, in_channels=256, out_channels=256, stride=1, kernel_size=2, padding_mode='replicate')
    # x = self.identity_block(x=x, in_channels=256, out_channels=256, stride=1, kernel_size=2, padding_mode='replicate')
    # x = self.identity_block(x=x, in_channels=256, out_channels=256, stride=1, kernel_size=2, padding_mode='replicate')
    # x = self.identity_block(x=x, in_channels=256, out_channels=256, stride=1, kernel_size=2, padding_mode='replicate')
    # #####################################################################
    # # Residual Block
    # #####################################################################


    # #####################################################################
    # # Encoder
    # #####################################################################
    x = self.deconv_block(x=x, in_channels=256, out_channels=256, kernel_size=2, stride=1, padding_mode='zeros')
    x = self.deconv_block(x=x, in_channels=256, out_channels=128, kernel_size=2, stride=2, padding_mode='zeros')
    x = self.deconv_block(x=x, in_channels=128, out_channels=64 , kernel_size=2, stride=2, padding_mode='zeros')
    # #####################################################################
    # # Encoder
    # #####################################################################

    x = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=2, stride=1, padding_mode='zeros')(x)
    x = F.tanh(x)

    return x


In [0]:
model = Model(3)

In [0]:
image = model(torch.rand((1,3,256,256)));
im_show(image);

print(image.shape)

## Loss

In [0]:
def custom_loss(pred, true):
  loss = nn.MSELoss()
  return loss(pred, true)

## Optimizer

In [0]:
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Train

In [0]:
def checkpoint(model, dir):
  return torch.utils.checkpoint.checkpoint(model)

def train_step():
  return

def train_loop():
  for epoch in range(EPOCHS):
    for img in images:
      model.zero_grad()

      prediction = model("INSET MODEL INPUTS HERE")
      
      loss = custom_loss(prediction, img)
      loss.backward()

      optimizer.step()
    print("Loss: {}".format(loss))
    with torch.no_grad():

train_loop()

# Evaluate

In [0]:
with torch.no_grad():
  for image in images:
    prediction = net(image.view(-1, IMG_WIDTH*IMG_HEIGHT))
    print(loss(prediction, image))

# Save model

In [0]:
def save_model(dir, model):
  torch.save(model.state_dict(), dir)

save_model(CHECKPOINT_DIR, net)

# Load Model

In [0]:
net = Net()
net.load_state_dict(torch.load(CHECKPOINT_DIR))